In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_17903/1527706816.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
conn = psycopg2.connect(host = '143.248.248.192', dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select\
                      x.q_id\
                    , x.q_posttypeid\
                    , x.q_acceptedanswerid\
                    , x.q_parentid\
                    , x.q_creationdate\
                    , x.q_score\
                    , x.q_viewcount\
                    , x.q_owneruserid\
                    , x.q_title\
                    , x.q_tags\
                    , x.q_answercount\
                    , x.q_commentcount\
                    , x.q_reputation\
                    , f.body             as q_body\
                    , b.id               as a_id\
                    , b.posttypeid       as a_posttypeid\
                    , b.acceptedanswerid as a_acceptedanswerid\
                    , b.parentid         as a_parentid\
                    , b.creationdate     as a_creationdate\
                    , b.score            as a_score\
                    , b.viewcount        as a_viewcount\
                    , b.owneruserid      as a_owneruserid\
                    , b.title            as a_title\
                    , b.tags             as a_tags\
                    , b.answercount      as a_answercount\
                    , b.commentcount     as a_commentcount\
                    , d.reputation       as a_reputation\
                    , g.body             as a_body\
      from (\
               select a.id               as q_id\
                    , a.posttypeid       as q_posttypeid\
                    , a.acceptedanswerid as q_acceptedanswerid\
                    , a.parentid         as q_parentid\
                    , a.creationdate     as q_creationdate\
                    , a.score            as q_score\
                    , a.viewcount        as q_viewcount\
                    , a.owneruserid      as q_owneruserid\
                    , a.title            as q_title\
                    , a.answercount      as q_answercount\
                    , a.commentcount     as q_commentcount\
                    , c.reputation       as q_reputation\
                    , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags\
               from posts a\
                  , users c\
               where a.creationdate >= '2023-11-17'\
                 and a.posttypeid = '1'\
                 and a.owneruserid is not null\
                 and c.id = a.owneruserid\
                 and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %')\
           )   x\
            , posts b\
            , users d\
            , postsbody f\
            , postsbody g\
    where b.parentid = x.q_id\
      and b.posttypeid = '2'\
      and b.owneruserid is not null\
      and d.id = b.owneruserid\
      and x.q_id = f.id \
      and b.id = g.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

column a.text does not exist
LINE 1: ...eputation       as q_reputation                 , a.text    ...
                                                             ^



In [4]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'q_text' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
, 'a_text' 
])

NameError: name 'rows' is not defined

In [ ]:
df.head()

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
0,77336626,1,NaN,None,2023-10-21 15:19:04.530,0,34,724835,Why is @Enitity required for @KeySpace enitity...,spring-data spring-data-keyvalue,...,77336626,2023-10-24 11:10:55.103,1,None,724835,None,None,None,0,4903
1,77330927,1,NaN,None,2023-10-20 12:28:41.247,0,52,22773792,How to retrieve spring datasource properties f...,java spring spring-boot environment-variables,...,77330927,2023-10-24 11:20:40.770,0,None,12689021,None,None,None,0,21
2,77351623,1,NaN,None,2023-10-24 11:20:18.453,-2,51,16497016,Why does Integer.parseInt produce Exception in...,java pattern-matching text-parsing,...,77351623,2023-10-24 11:23:15.500,2,None,1180351,None,None,None,0,7014
3,77350630,1,77350938.0,None,2023-10-24 08:40:37.300,0,31,20500971,Spring Boot - @ConfigurationProperties with ne...,java spring-boot properties yaml mapping,...,77350630,2023-10-24 09:31:52.397,2,None,6633045,None,None,None,0,486
4,77328840,1,NaN,None,2023-10-20 06:52:53.487,-1,50,8196323,Endless recursion when trying to save an entit...,java sql spring spring-boot jpa,...,77328840,2023-10-24 12:26:25.740,0,None,12689021,None,None,None,0,21


In [ ]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

2023-08-03 01:04:49.693000
2023-12-03 10:02:09.807000


In [ ]:
# 전체 데이터셋 
with open('../../data/bert_df.pkl', 'wb') as f:
    pickle.dump(df, f)